In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
import os
# using python csv library for writing results
import csv
# loading data in windows environment
import pathlib
import re
print (tf.__version__)
from kaggle_datasets import KaggleDatasets
import PIL.Image
import cv2

# Hardware platform: You may want to scale your training onto multiple GPUs on one machine, 
# or multiple machines in a network (with 0 or more GPUs each), or on Cloud TPUs.
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None


if tpu:
    # Passing in the name of the CloudTPU
    tf.config.experimental_connect_to_cluster(tpu)
    # The TPU initialization code has to be at the beginning of the program
    tf.tpu.experimental.initialize_tpu_system(tpu)
    # A distribution strategy is an abstraction that can be used to drive models on CPU, GPUs or TPUs
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.experimental.CentralStorageStrategy()
    print ("GPU VERSION")


# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
GCS_DS_PATH = KaggleDatasets().get_gcs_path('cassava-leaf-disease-classification')
GCS_PATH = GCS_DS_PATH + '/train_tfrecords'
#TEST_GCS_PATH = GCS_DS_PATH + '/test_tfrecords'
AUTO = tf.data.experimental.AUTOTUNE

#TRAINING_FILENAMES = tf.io.gfile.glob("../input/cassava-leaf-disease-classification/" + 'train_tfrecords/*.tfrec')
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/*.tfrec')

# Image height after resizing
IMG_HEIGHT=386
# Image width after resizing
IMG_WIDTH=386

# Helper Function 
def mapping_image(dat):
    # Extracting image content from tfrecords
    TFREC_MAP = {"image": tf.io.FixedLenFeature([], tf.string)}
    # Parsing the extracted content
    parsed_dat = tf.io.parse_single_example(dat, TFREC_MAP)
    # Decoding the image
    image = tf.io.decode_image(parsed_dat['image'], channels=3)
    # Normalizing the image
    image = tf.cast(image/255, dtype=tf.float32)
    return image

# Helper Function
def mapping_id(dat):
    # Extracting targeting label from tfrecords
    TFREC_MAP = {"target": tf.io.FixedLenFeature([], tf.int64)}
    # Parsing the targeting label
    parsed_dat = tf.io.parse_single_example(dat, TFREC_MAP)
    # Loading image label
    image_id = parsed_dat['target']
    # Casting image label into integer
    image_id = tf.cast(image_id, dtype=tf.int32)
    return image_id

# Helper Function
def mapping_name(dat):
    # Extracting targeting name from tfrecords
    TFREC_MAP = {"image_name": tf.io.FixedLenFeature([], tf.string)}
    # Parsing the image name
    parsed_dat = tf.io.parse_single_example(dat, TFREC_MAP)
    # Loading the image name
    image_name = parsed_dat['image_name']
    return image_name

# Helper Function
def mapping_image_id(dat):
    # Extracting image and targeting label from tfrecords
    TFREC_MAP = {"image": tf.io.FixedLenFeature([], tf.string),
                 "target": tf.io.FixedLenFeature([], tf.int64),
                }
    # Parsing the extracted data
    parsed_dat = tf.io.parse_single_example(dat, TFREC_MAP)
    # Decoding the image data
    image = tf.io.decode_jpeg(parsed_dat['image'], channels=3)
    # Normalizing image data
    image = tf.cast(image/255, dtype=tf.float32)
    # Loading the predicting label
    target = parsed_dat['target']
    # Casting the label into integer
    target = tf.cast(target, dtype=tf.int32)
    return image, target

# Processing Input Training Data
train_data_set = tf.data.TFRecordDataset(TRAINING_FILENAMES, num_parallel_reads=AUTO)
# Extracting the Training Data
train_data = train_data_set.map(mapping_image_id)
# Shuffling the Training Data
train_data = train_data.shuffle(1000, reshuffle_each_iteration=False)

# TPU Training
STEPS_PER_EPOCH = 32

# Number of iteration steps
EPOCHS = 10

# Image list
IMG=[]
# Label list
LAB=[]

# Generating Image and Labeling for Training
L=0
# Scaning through the training dataset
for dat, lab in train_data:
    # Loading the first 16000 images for training
    if (L>1000):
        break
    # Resizing the training image
    re_image = tf.image.resize_with_pad(dat, IMG_HEIGHT,IMG_WIDTH, method=tf.image.ResizeMethod.BICUBIC,antialias=True)
    # Generating Input Image
    IMG.append(re_image)
    # Generating Input Label
    LAB.append(lab)
    #print (L)
    L=L+1

# Inspecting the training image
# Plotting the figure of 8X8
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot()
ax.imshow(IMG[0])
plt.show()

# Converting Image Data into Tensor Object
IMG = tf.convert_to_tensor(IMG)
# Converting Label Data into Tensor Object
LAB = tf.convert_to_tensor(LAB)

# TPU training loop
with strategy.scope():
    # Sequential Model
    model = tf.keras.Sequential()
    # Loading pretrained model
    tensor_model = tf.keras.applications.DenseNet201(input_shape=[IMG_HEIGHT, IMG_WIDTH, 3], include_top=False, weights='imagenet')
    # Freeze the layers
    tensor_model.trainable = False
    # Getting number of model layers
    print ("TOTAL LAYER {}".format(len(tensor_model.layers)))
    # Adding the pretrained model to the stack
    model.add(tensor_model)
    # Global Average Pooling
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    # Final decision layer
    model.add(tf.keras.layers.Dense(5))
    # Model summary
    model.summary()
    # Applying the Sparse Categorical Cross Entropy as the loss function
    model_loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # Gradiant Descent Optimizer 
    model_opt_func = tf.keras.optimizers.SGD(lr=0.00001)
    # Computing the weighted mean value
    metrics = tf.keras.metrics.Mean()
    # Computes the crossentropy metric between the labels and predictions.
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    # Compiling the model
    model.compile(optimizer=model_opt_func,loss=model_loss_func,metrics=[accuracy])


# Train the Model
# Saving the model
# Storing in Dataset Kaggle
history = model.fit(x=IMG,y=LAB, epochs=EPOCHS, steps_per_epoch=STEPS_PER_EPOCH)
model.save('kaggle_cassava_model_Dense201.h5')